In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from math import comb
from pathlib import Path
import json
import os

from coderm.eval.metrics import get_pass_ks
from coderm.utils import gunzip_json_read

In [ ]:
log_directories = [
    # os.path.join("p4o_results", "idea_filter_4omi200_temp0.8"),
    # os.path.join("p4o_results", "cobs_4omi_temp0.8"),
    os.path.join("p4o_results", "simple_filter_4omi1_temp0")
]

data_list = [gunzip_json_read(log_dir) for log_dir in log_directories]

def analyze_data(data):
    num_codes = 0
    num_empty = 0
    num_timeout = 0
    num_fst = 0
    for item in data["items"]:
        for code in item["results"]:
            num_codes += 1
            if code["passing"] == True:
                continue
            if code["code"] == "" or code["code"] == "# No successful generation\n":
                num_empty += 1
            elif "errored with 'Timeout'" in code["output"]:
                num_timeout += 1
            else:
                num_fst += 1
    num_correct = num_codes - (num_empty + num_timeout + num_fst)
    return num_correct, num_empty, num_timeout, num_fst, num_codes

results = [analyze_data(data) for data in data_list]

# Data for the pie chart
labels = ['Correct', 'Empty', 'Timeout', 'Failed']
colors = ['#4CAF50', '#FF9800', '#F44336', '#2196F3']
explode = (0.1, 0, 0, 0)  # explode the 1st slice (Correct)

# Plotting the pie charts
fig, axes = plt.subplots(1, len(results), figsize=(16, 8))

if len(results) == 1:
    axes = [axes]  # Ensure axes is iterable

for ax, (num_correct, num_empty, num_timeout, num_fst, num_codes), log_dir in zip(axes, results, log_directories):
    sizes = [num_correct, num_empty, num_timeout, num_fst]
    ax.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=140)
    ax.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
    ax.set_title(f'Distribution of Code Results ({os.path.basename(log_dir)})')

plt.show()